In [1]:
import json, os
from datasets import load_dataset

In [3]:
SPLIT = "train"
SAVE_FILEPATH = "[FILEPATH]"
ds = load_dataset("json", data_files=f"{SPLIT}_revised.json", keep_in_memory=True)["train"]

In [5]:
def replace(example, v_to_be_replaced, v_replacing):
    example["vertexSet"][v_replacing].extend(example["vertexSet"][v_to_be_replaced])
    del example["vertexSet"][v_to_be_replaced]
    if v_replacing > v_to_be_replaced:
        v_replacing -= 1
    for r_id in range(len(example["labels"])):
        if example["labels"][r_id]['h'] == v_to_be_replaced:
            example["labels"][r_id]['h'] = v_replacing
        elif example["labels"][r_id]['h'] > v_to_be_replaced:
            example["labels"][r_id]['h'] -= 1
        if example["labels"][r_id]['t'] == v_to_be_replaced:
            example["labels"][r_id]['t'] = v_replacing
        elif example["labels"][r_id]['t'] > v_to_be_replaced:
            example["labels"][r_id]['t'] -= 1
    return example

In [6]:
from tqdm.auto import tqdm

In [7]:
def remove_alias(example, alias_id, v_to_be_replaced, v_replacing):
    del example["vertexSet"][v_to_be_replaced][alias_id]
    vertex_del = False
    if len(example["vertexSet"][v_to_be_replaced]) == 0:
        del example["vertexSet"][v_to_be_replaced]
        if v_replacing > v_to_be_replaced:
            v_replacing -= 1
        for r_id in range(len(example["labels"])):
            if example["labels"][r_id]['h'] == v_to_be_replaced:
                example["labels"][r_id]['h'] = v_replacing
            elif example["labels"][r_id]['h'] > v_to_be_replaced:
                example["labels"][r_id]['h'] -= 1
            if example["labels"][r_id]['t'] == v_to_be_replaced:
                example["labels"][r_id]['t'] = v_replacing
            elif example["labels"][r_id]['t'] > v_to_be_replaced:
                example["labels"][r_id]['t'] -= 1
        vertex_del = True
    return example, vertex_del

def overlap(a_1, a_2):
    if a_1["sent_id"] == a_2["sent_id"]:
        if (a_1["pos"][0] <= a_2["pos"][0] and a_2["pos"][0] < a_1["pos"][1]) or (a_2["pos"][0] <= a_1["pos"][0] and a_1["pos"][0] < a_2["pos"][1]):
            if a_1["pos"][1] - a_1["pos"][0] > a_2["pos"][1] - a_2["pos"][0]:
                return 1
            return 2
    return 0

In [8]:
new_examples = []
for i in tqdm(range(len(ds))):
    if i == 872:
        pass
    example = ds[i]
    vs_1_id = 0
    vs_2_id = 0
    while vs_1_id < len(example["vertexSet"]) and vs_2_id < len(example["vertexSet"]):
        if vs_1_id < vs_2_id:
            aliases_1 = set([x["name"] for x in example["vertexSet"][vs_1_id]])
            aliases_2 = set([x["name"] for x in example["vertexSet"][vs_2_id]])
            if len(aliases_1.intersection(aliases_2)) > 0:
                if len(example["vertexSet"][vs_1_id]) < len(example["vertexSet"][vs_2_id]):
                    example = replace(example, vs_1_id, vs_2_id)
                else:
                    example = replace(example, vs_2_id, vs_1_id)
            else:
                vs_2_id += 1
                if vs_2_id == len(example["vertexSet"]):
                    vs_1_id += 1
                    vs_2_id = 0
        else:
            vs_2_id += 1
            if vs_2_id == len(example["vertexSet"]):
                vs_1_id += 1
                vs_2_id = 0

    vs_1_id = 0
    vs_2_id = 0
    while vs_1_id < len(example["vertexSet"]) and vs_2_id < len(example["vertexSet"]):
        if vs_1_id <= vs_2_id:
            aid_1 = 0
            aid_2 = 0
            delta = False
            while aid_1 < len(example["vertexSet"][vs_1_id]) and aid_2 < len(example["vertexSet"][vs_2_id]):
                overlap_res = overlap(example["vertexSet"][vs_1_id][aid_1], example["vertexSet"][vs_2_id][aid_2])
                if overlap_res == 0 or (vs_1_id == vs_2_id and aid_1 <= aid_2):
                    aid_1 += 1
                    if aid_1 == len(example["vertexSet"][vs_1_id]):
                        aid_1 = 0
                        aid_2 += 1
                    continue
                if overlap_res == 1:
                    example, vertex_del = remove_alias(example, aid_2, vs_2_id, vs_1_id)
                    delta = True
                else:
                    example, vertex_del = remove_alias(example, aid_1, vs_1_id, vs_2_id)
                    delta = True
                if vertex_del:
                    break
            if not delta:
                vs_2_id += 1
                if vs_2_id == len(example["vertexSet"]):
                    vs_1_id += 1
                    vs_2_id = 0
        else:
            vs_2_id += 1
            if vs_2_id == len(example["vertexSet"]):
                vs_1_id += 1
                vs_2_id = 0
    
    new_examples.append(example)

  0%|          | 0/500 [00:00<?, ?it/s]

In [9]:
for ex in new_examples:
    vs_count = len(ex["vertexSet"])
    assert vs_count > 0
    for vs in ex["vertexSet"]:
        assert len(vs) > 0
    for rel in ex["labels"]:
        assert rel["h"] < vs_count
        assert rel["t"] < vs_count

In [10]:
with open(os.path.join(SAVE_FILEPATH, f"{SPLIT}_revised_entFix.json"), "w") as f:
    json.dump(new_examples, f)